### Online Prediction

Copy data for making the predictions.

In [1]:
# Some code to determine a unique bucket name for the purposes of the sample
from gcp.context import Context

CLOUD_PROJECT = Context.default().project_id
ml_bucket_name = CLOUD_PROJECT + '-mldata'
ml_bucket_path = 'gs://' + ml_bucket_name

INPUT_DIR = ml_bucket_path + '/sampledata/ml/image/input'
OUTPUT_DIR = ml_bucket_path + '/sampledata/ml/image/output'

In [3]:
%%bash -s "$INPUT_DIR" "$OUTPUT_DIR"
gsutil cp $1/daisy/6884975451_c74f445d69_m.jpg /datalab/ml/predict/img.jpg
gsutil cp $1/classify_image_graph_def.pb /datalab/ml/predict/classify_image_graph_def.pb
gsutil cp $2/dict.txt /datalab/ml/predict/dict.txt

Copying gs://cloud-ml-users-mldata/sampledata/ml/image/input/daisy/6884975451_c74f445d69_m.jpg...
Copying gs://cloud-ml-users-mldata/sampledata/ml/image/input/classify_image_graph_def.pb...
Copying gs://cloud-ml-users-mldata/sampledata/ml/image/output/dict.txt...


Like training, we also need to preprocess the image. But we'll do it locally, using the same preprocess.py but as a library.
(Ignore the warning message "AssertionError() in <bound method InteractiveSession.__del__...")

In [4]:
import preprocess
import re
from google.protobuf import json_format

preprocessor = preprocess.LocalImageTransformer('/datalab/ml/predict')
with preprocess._open('/datalab/ml/predict/img.jpg') as f:
  example = preprocessor.preprocess_list([f.read()], ['/datalab/ml/predict/img.jpg'])[0]
  example = re.sub(r'\s+', ' ', example)
  

Let's predict with the cloud model we trained.

In [5]:
%%ml predict -m image.v1 --cloud  -p -d example

,label,score
0,4,0.97895


The label is a number because our preprocessing has mapped it to an integer. We can figure it out by dumping the label file.

In [7]:
with open('/datalab/ml/predict/dict.txt') as f:
  labels = f.readlines()
labels = [e.strip() for e in labels]
print labels

['tulips', 'roses', 'dandelion', 'sunflowers', 'daisy']


Labels are 0 based, so it is daisy!

In [8]:
%%bash
cp /datalab/ml/predict/img.jpg ./

In [9]:
from IPython.core.display import HTML
HTML('<div style="padding: 16px"><img src="%s" /><span>%s (%.2f)</span></div>' % ('img.jpg', labels[4], 0.982618))

### Batch Prediction

Define prediction data first.

In [44]:
CLOUD_TEST_DATA = '%s/data/data.test.json-00000-of-00010' % OUTPUT_DIR

In [45]:
%%ml predict -m image.v1 -b batch --cloud
$CLOUD_TEST_DATA